In [69]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime


In [70]:
data_usd_tr = pd.read_csv("csv files/USD_TRY.csv")
data_usd_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2870 entries, 0 to 2869
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tarih   2870 non-null   object
 1   Şimdi   2870 non-null   object
 2   Açılış  2870 non-null   object
 3   Yüksek  2870 non-null   object
 4   Düşük   2870 non-null   object
 5   Hac.    672 non-null    object
 6   Fark %  2870 non-null   object
dtypes: object(7)
memory usage: 157.1+ KB


In [71]:
df = data_usd_tr.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)

In [72]:
# We need to change from "," to "." because it will be a float.
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '.')))
df

,Tarih,Şimdi
0,30.12.2022,18.6883
1,29.12.2022,18.6941
2,28.12.2022,18.7157
3,27.12.2022,18.6669
4,26.12.2022,18.6618
...,...,...
2865,06.01.2012,1.8776
2866,05.01.2012,1.8830
2867,04.01.2012,1.8785
2868,03.01.2012,1.8730


## LSTM

In [73]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[18.6883 18.6941 18.7157 18.6669 18.6618] 18.6637
[18.6941 18.7157 18.6669 18.6618 18.6637] 18.6424
[18.7157 18.6669 18.6618 18.6637 18.6424] 18.6749
[18.6669 18.6618 18.6637 18.6424 18.6749] 18.6479
[18.6618 18.6637 18.6424 18.6749 18.6479] 18.6426
[18.6637 18.6424 18.6749 18.6479 18.6426] 18.6343
[18.6424 18.6749 18.6479 18.6426 18.6343] 18.6368
[18.6749 18.6479 18.6426 18.6343 18.6368] 18.6232
[18.6479 18.6426 18.6343 18.6368 18.6232] 18.575
[18.6426 18.6343 18.6368 18.6232 18.575 ] 18.6325
[18.6343 18.6368 18.6232 18.575  18.6325] 18.6503
[18.6368 18.6232 18.575  18.6325 18.6503] 18.6324
[18.6232 18.575  18.6325 18.6503 18.6324] 18.6355
[18.575  18.6325 18.6503 18.6324 18.6355] 18.624
[18.6325 18.6503 18.6324 18.6355 18.624 ] 18.6227
[18.6503 18.6324 18.6355 18.624  18.6227] 18.6303
[18.6324 18.6355 18.624  18.6227 18.6303] 18.6313
[18.6355 18.624  18.6227 18.6303 18.6313] 18.5895
[18.624  18.6227 18.6303 18.6313 18.5895] 18.6374
[18.6227 18.6303 18.6313 18.5895 18.6374] 18.6166
[1

In [74]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
90/90 [==============================] - 2s 5ms/step - loss: 4.4366
Epoch 2/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0452
Epoch 3/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0467
Epoch 4/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0446
Epoch 5/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0458
Epoch 6/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0437
Epoch 7/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0401
Epoch 8/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0427
Epoch 9/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0397
Epoch 10/200
90/90 [==============================] - 1s 6ms/step - loss: 0.0359
Epoch 11/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0357
Epoch 12/200
90/90 [==============================] - 0s 5ms/step - loss: 0.0374
Epoch 13/200
90/90 [=================

## Daily


In [75]:
x = datetime.datetime.now()
print("Prediction:", round(yhat[0][0],2), "₺ USD/TL in", x.strftime('%x')) 

Prediction: 18.73 ₺ USD/TL in 01/01/23


## Weekly

In [76]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi']
                [2], df['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi']
                [2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],
                float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_4),
                float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)
yhat_2 = float(yhat_2)
yhat_3 = float(yhat_3)
yhat_4 = float(yhat_4)
yhat_5 = float(yhat_5)


In [77]:
# onluk olarak yukarı yuvarlandı.
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)} ₺ Usd/TRY")

Monday: 18.73 Tuesday : 18.818 Wednesday: 18.761 Thursday: 18.767 Friday: 18.763 ₺ Usd/TRY


## Monthly

In [78]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [79]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[18.6883 18.6637 18.6343 18.6503 18.6303] 18.585100173950195
[18.6637 18.6343 18.6503 18.6303 18.5851] 18.588899612426758
[18.6343 18.6503 18.6303 18.5851 18.5889] 18.570499420166016
[18.6503 18.6303 18.5851 18.5889 18.5705] 18.56570053100586
[18.6303 18.5851 18.5889 18.5705 18.5657] 18.581300735473633
[18.5851 18.5889 18.5705 18.5657 18.5813] 18.58839988708496
[18.5889 18.5705 18.5657 18.5813 18.5884] 18.558500289916992
[18.5705 18.5657 18.5813 18.5884 18.5585] 18.562000274658203
[18.5657 18.5813 18.5884 18.5585 18.562 ] 18.5
[18.5813 18.5884 18.5585 18.562  18.5   ] 18.41309928894043
[18.5884 18.5585 18.562  18.5    18.4131] 18.25429916381836
[18.5585 18.562  18.5    18.4131 18.2543] 18.233400344848633
[18.562  18.5    18.4131 18.2543 18.2334] 18.17770004272461
[18.5    18.4131 18.2543 18.2334 18.1777] 18.143299102783203
[18.4131 18.2543 18.2334 18.1777 18.1433] 18.084999084472656
[18.2543 18.2334 18.1777 18.1433 18.085 ] 17.93239974975586
[18.2334 18.1777 18.1433 18.085  17.9324] 17

In [80]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
18/18 [==============================] - 1s 6ms/step - loss: 11.1903
Epoch 2/300
18/18 [==============================] - 0s 6ms/step - loss: 0.9751
Epoch 3/300
18/18 [==============================] - 0s 6ms/step - loss: 0.3529
Epoch 4/300
18/18 [==============================] - 0s 6ms/step - loss: 0.1519
Epoch 5/300
18/18 [==============================] - 0s 6ms/step - loss: 0.1878
Epoch 6/300
18/18 [==============================] - 0s 6ms/step - loss: 0.1731
Epoch 7/300
18/18 [==============================] - 0s 6ms/step - loss: 0.1645
Epoch 8/300
18/18 [==============================] - 0s 6ms/step - loss: 0.2292
Epoch 9/300
18/18 [==============================] - 0s 6ms/step - loss: 0.1605
Epoch 10/300
18/18 [==============================] - 0s 6ms/step - loss: 0.1621
Epoch 11/300
18/18 [==============================] - 0s 6ms/step - loss: 0.1581
Epoch 12/300
18/18 [==============================] - 0s 6ms/step - loss: 0.2391
Epoch 13/300
18/18 [================

In [81]:
print(f"Next Month -> Prediction: {round(float(yhat[0][0]),3)}  ₺ USD/TRY in {x.strftime('%B')} ")

Next Month -> Prediction: 18.556  ₺ USD/TRY in January 
